In [1]:
import pandas as pd
import numpy as np
import datetime, warnings, scipy
import seaborn as sns
import matplotlib as mpl

In [2]:
df1 = pd.read_csv(filepath_or_buffer = '../data/2008.csv')

### CRSDepTime is Scheduled Departure Time

In [51]:
df1.loc[:, "CRSDepTime"]

0          1955
1           735
2           620
3           930
4          1755
5          1915
6          1830
7          1040
8           615
9          1620
10          700
11         1510
12         1430
13          715
14         1700
15         1020
16         1425
17          745
18         1255
19         1325
20          705
21         1625
22         1840
23         1030
24          800
25         1455
26         1255
27         1925
28          635
29          730
           ... 
7009698    1635
7009699    1221
7009700    1845
7009701    1500
7009702    1522
7009703    1910
7009704    1445
7009705     830
7009706    1440
7009707    1755
7009708     710
7009709    1520
7009710    1220
7009711    1041
7009712     843
7009713     815
7009714     545
7009715     850
7009716     936
7009717     600
7009718     847
7009719     640
7009720     800
7009721     615
7009722     750
7009723     959
7009724     835
7009725     700
7009726    1240
7009727    1103
Name: CRSDepTime, Length

In [52]:
#Convert the HHMM format to HH:MM format
def format_hour(hm):
    if pd.isnull(hm):
        return np.nan
    else:
        if hm == 2400: hm = 0
        hm = "{0:04d}".format(int(hm))
        newHM = datetime.time(int(hm[0:2]), int(hm[2:4]))
        return newHM

In [53]:
df1['CRSDepTime_Formatted'] = df1['CRSDepTime'].apply(format_hour)

In [54]:
df1['CRSDepTime_Formatted']

0          19:55:00
1          07:35:00
2          06:20:00
3          09:30:00
4          17:55:00
5          19:15:00
6          18:30:00
7          10:40:00
8          06:15:00
9          16:20:00
10         07:00:00
11         15:10:00
12         14:30:00
13         07:15:00
14         17:00:00
15         10:20:00
16         14:25:00
17         07:45:00
18         12:55:00
19         13:25:00
20         07:05:00
21         16:25:00
22         18:40:00
23         10:30:00
24         08:00:00
25         14:55:00
26         12:55:00
27         19:25:00
28         06:35:00
29         07:30:00
             ...   
7009698    16:35:00
7009699    12:21:00
7009700    18:45:00
7009701    15:00:00
7009702    15:22:00
7009703    19:10:00
7009704    14:45:00
7009705    08:30:00
7009706    14:40:00
7009707    17:55:00
7009708    07:10:00
7009709    15:20:00
7009710    12:20:00
7009711    10:41:00
7009712    08:43:00
7009713    08:15:00
7009714    05:45:00
7009715    08:50:00
7009716    09:36:00


### Apply the same format for Scheduled Arrival Time, Departure Time, Arrival Time

In [55]:
df1['CRSArrTime_Formatted'] = df1['CRSArrTime'].apply(format_hour)
df1['CRSArrTime_Formatted']

0          22:25:00
1          10:00:00
2          07:50:00
3          11:00:00
4          19:25:00
5          21:10:00
6          19:40:00
7          11:50:00
8          06:50:00
9          16:55:00
10         09:15:00
11         17:25:00
12         14:25:00
13         07:10:00
14         16:55:00
15         10:10:00
16         16:25:00
17         09:55:00
18         15:10:00
19         14:35:00
20         08:10:00
21         17:35:00
22         19:50:00
23         11:40:00
24         09:10:00
25         16:05:00
26         16:10:00
27         22:35:00
28         09:45:00
29         10:20:00
             ...   
7009698    17:58:00
7009699    13:59:00
7009700    20:06:00
7009701    16:42:00
7009702    18:23:00
7009703    20:16:00
7009704    16:22:00
7009705    10:08:00
7009706    17:04:00
7009707    20:15:00
7009708    08:37:00
7009709    17:18:00
7009710    15:52:00
7009711    13:03:00
7009712    10:21:00
7009713    15:26:00
7009714    06:50:00
7009715    10:05:00
7009716    11:19:00


### DepTime and ArrTime is the real Departure Time and Arrival Time

In [56]:
df1['DepTime_Formatted'] = df1['DepTime'].apply(format_hour)

In [57]:
df1['ArrTime_Formatted'] = df1['ArrTime'].apply(format_hour)

## Putting Scheduled Departure Time, Scheduled Arrival Time, Departure Time and Arrival Time togerther

In [4]:
#df1.loc[:, ['CRSDepTime_Formatted', 'CRSArrTime_Formatted', 'DepTime_Formatted', 'ArrTime_Formatted']]

### Combine DayOfMonth, Month, Year

In [59]:
df1['Day'] = df1['DayofMonth']

In [60]:
df1['DepDate'] = pd.to_datetime(df1[['Year','Month', 'Day']])

In [61]:
def date_time_combine(r):
    if pd.isnull(r[0]) or pd.isnull(r[1]):
        return np.nan
    else:
        return datetime.datetime.combine(r[0], r[1])
def create_scheduled_dep_time(df, col):
    dt = []
    for index, cols in df1[['DepDate', col]].iterrows():
        if pd.isnull(cols[1]):
            dt.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days = 1)
            cols[1] = datetime.time(0,0)
            dt.append(date_time_combine(cols))
        else:
            cols[1] = format_hour(cols[1])
            dt.append(date_time_combine(cols))
    return pd.Series(dt)

### Combine the scheduled departure time and departure date

In [62]:
df1['CRSDep_Date_Time'] = create_scheduled_dep_time(df1, 'CRSDepTime')

### CRSDep_Date_Time
This feature shows the **scheduled departure time** in the format of Date and Time 

In [63]:
df1['CRSDep_Date_Time']

0         2008-01-03 19:55:00
1         2008-01-03 07:35:00
2         2008-01-03 06:20:00
3         2008-01-03 09:30:00
4         2008-01-03 17:55:00
5         2008-01-03 19:15:00
6         2008-01-03 18:30:00
7         2008-01-03 10:40:00
8         2008-01-03 06:15:00
9         2008-01-03 16:20:00
10        2008-01-03 07:00:00
11        2008-01-03 15:10:00
12        2008-01-03 14:30:00
13        2008-01-03 07:15:00
14        2008-01-03 17:00:00
15        2008-01-03 10:20:00
16        2008-01-03 14:25:00
17        2008-01-03 07:45:00
18        2008-01-03 12:55:00
19        2008-01-03 13:25:00
20        2008-01-03 07:05:00
21        2008-01-03 16:25:00
22        2008-01-03 18:40:00
23        2008-01-03 10:30:00
24        2008-01-03 08:00:00
25        2008-01-03 14:55:00
26        2008-01-03 12:55:00
27        2008-01-03 19:25:00
28        2008-01-03 06:35:00
29        2008-01-03 07:30:00
                  ...        
7009698   2008-12-13 16:35:00
7009699   2008-12-13 12:21:00
7009700   

### Putting all the necessary features together
In this model, we will consider these following features: 
- UniqueCarrier
- Origin
- Dest
- DepTime_Formatted
- DepDelay
- CRSArrTime_Formatted
- ArrTime_Formatted
- ArrDelay: Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers.
- ActualElapsedTime
- CRSElapsedTime

In [3]:
notUsedFeatures = ['TaxiIn', 'TaxiOut', 'Year', 'Month', 'DayofMonth', 'DayOfWeek', 'NASDelay', 'WeatherDelay', 'CarrierDelay', 'SecurityDelay', 'LateAircraftDelay', 'Cancelled', 'CancellationCode', 'Diverted', 'AirTime', 'FlightNum', 'TailNum']
df1.drop(notUsedFeatures, axis = 1, inplace=True)
# df1 = df1[['UniqueCarrier', 'Origin', 'Dest', 'DepTime', 'DepDelay', 'CRSArrTime_Formatted', 'ArrTime_Formatted', 'ArrDelay', 'CRSElapsedTime', 'ActualElapsedTime', 'Distance']]
df1 = df1[['UniqueCarrier', 'Origin', 'Dest','CRSDepTime', 'DepTime', 'DepDelay', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'CRSElapsedTime', 'ActualElapsedTime', 'Distance']]

In [4]:
df1

,UniqueCarrier,Origin,Dest,CRSDepTime,DepTime,DepDelay,CRSArrTime,ArrTime,ArrDelay,CRSElapsedTime,ActualElapsedTime,Distance
0,WN,IAD,TPA,1955,2003.0,8.0,2225,2211.0,-14.0,150.0,128.0,810
1,WN,IAD,TPA,735,754.0,19.0,1000,1002.0,2.0,145.0,128.0,810
2,WN,IND,BWI,620,628.0,8.0,750,804.0,14.0,90.0,96.0,515
3,WN,IND,BWI,930,926.0,-4.0,1100,1054.0,-6.0,90.0,88.0,515
4,WN,IND,BWI,1755,1829.0,34.0,1925,1959.0,34.0,90.0,90.0,515
5,WN,IND,JAX,1915,1940.0,25.0,2110,2121.0,11.0,115.0,101.0,688
6,WN,IND,LAS,1830,1937.0,67.0,1940,2037.0,57.0,250.0,240.0,1591
7,WN,IND,LAS,1040,1039.0,-1.0,1150,1132.0,-18.0,250.0,233.0,1591
8,WN,IND,MCI,615,617.0,2.0,650,652.0,2.0,95.0,95.0,451
9,WN,IND,MCI,1620,1620.0,0.0,1655,1639.0,-16.0,95.0,79.0,451


### Save preprocessed dataset 
Write the preprocessed dataset to csv file

In [26]:
df1.to_csv(path_or_buf='../data/preprocessed_2008.csv', index=False)

In [2]:
#Read the preprocessed data
df1 = pd.read_csv('../data/preprocessed_2008.csv')

In [13]:
df1

,UniqueCarrier,Origin,Dest,DepTime,DepDelay,CRSArrTime,ArrTime,ArrDelay,CRSElapsedTime,ActualElapsedTime,Distance
0,WN,IAD,TPA,2003.0,8.0,2225,2211.0,-14.0,150.0,128.0,810
1,WN,IAD,TPA,754.0,19.0,1000,1002.0,2.0,145.0,128.0,810
2,WN,IND,BWI,628.0,8.0,750,804.0,14.0,90.0,96.0,515
3,WN,IND,BWI,926.0,-4.0,1100,1054.0,-6.0,90.0,88.0,515
4,WN,IND,BWI,1829.0,34.0,1925,1959.0,34.0,90.0,90.0,515
5,WN,IND,JAX,1940.0,25.0,2110,2121.0,11.0,115.0,101.0,688
6,WN,IND,LAS,1937.0,67.0,1940,2037.0,57.0,250.0,240.0,1591
7,WN,IND,LAS,1039.0,-1.0,1150,1132.0,-18.0,250.0,233.0,1591
8,WN,IND,MCI,617.0,2.0,650,652.0,2.0,95.0,95.0,451
9,WN,IND,MCI,1620.0,0.0,1655,1639.0,-16.0,95.0,79.0,451


In [5]:
carriers = pd.read_csv('../data/carriers.csv')

In [6]:
carriers

,Code,Description
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.
5,09Q,"Swift Air, LLC"
6,0BQ,DCA
7,0CQ,ACM AIR CHARTER GmbH
8,0FQ,"Maine Aviation Aircraft Charter, LLC"
9,0GQ,"Inter Island Airways, d/b/a Inter Island Air"


In [18]:
carriers.loc[carriers['Code'] == 'WN']

,Code,Description
1388,WN,Southwest Airlines Co.


In [17]:
carriers_abbr_dict = carriers.set_index('Code')['Description'].to_dict()
carriers_abbr_dict['WN']

'Southwest Airlines Co.'

### Airports dataset
The airports dataset will show the real name of the airports following their location and IATA code

In [8]:
airports_df = pd.read_csv('../data/airports.csv')
airports_df

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
5,01M,Tishomingo County,Belmont,MS,USA,34.491667,-88.201111
6,02A,Gragg-Wade,Clanton,AL,USA,32.850487,-86.611453
7,02C,Capitol,Brookfield,WI,USA,43.087510,-88.177869
8,02G,Columbiana County,East Liverpool,OH,USA,40.673313,-80.641406
9,03D,Memphis Memorial,Memphis,MO,USA,40.447259,-92.226961


In [9]:
airports_abbr_dict = airports_df.set_index('iata').T.to_dict()

In [10]:
airports_abbr_dict['SFO']

{'airport': 'San Francisco International',
 'city': 'San Francisco',
 'country': 'USA',
 'lat': 37.61900194,
 'long': -122.3748433,
 'state': 'CA'}

### The get_stats function
The get_stats function will count the number of airlines that has departure delay and arrival delay, the maximum and minimum number of minutes they delay and get the mean number of minutes.
Below, we apply the get_stats function first for departure delay

In [11]:
def get_stats(group):
    return {'Min': group.min(), 'Max': group.max(),
            'Count': group.count(), 'Mean': group.mean()}

# Dataframe with statitical infos on each airline about departure delay:
dep_delay_stats = df1['DepDelay'].groupby(df1['UniqueCarrier']).apply(get_stats).unstack()
dep_delay_stats = dep_delay_stats.sort_values('Count')
dep_delay_stats

,Count,Max,Mean,Min
UniqueCarrier,,,,
AQ,7758.0,336.0,-1.397783,-61.0
HA,61263.0,963.0,0.455201,-534.0
F9,95462.0,817.0,5.919602,-25.0
AS,148984.0,947.0,6.848722,-79.0
OH,191173.0,960.0,11.536153,-70.0
B6,192903.0,976.0,12.653396,-70.0
YV,245824.0,607.0,12.000675,-92.0
9E,255223.0,1127.0,6.765860,-54.0
FL,259458.0,1206.0,9.262713,-62.0


### Apply get_stats
And now we will apply get_stats function to arrival delay

In [12]:
arr_delay_stats = df1['ArrDelay'].groupby(df1['UniqueCarrier']).apply(get_stats).unstack()
arr_delay_stats = arr_delay_stats.sort_values('Count')
arr_delay_stats

,Count,Max,Mean,Min
UniqueCarrier,,,,
AQ,7752.0,338.0,-2.888674,-90.0
HA,61212.0,1050.0,1.264409,-519.0
F9,95384.0,809.0,6.108247,-67.0
AS,148492.0,948.0,4.804346,-81.0
OH,190695.0,955.0,11.817468,-81.0
B6,192114.0,834.0,11.084184,-83.0
YV,245131.0,606.0,11.775181,-73.0
9E,254322.0,1143.0,4.111135,-66.0
FL,258713.0,1175.0,9.091375,-86.0


### Checking the number of flights for each airline based on the departure airport

In [22]:
carrier = 'WN'
carriers.loc[carriers['Code'] == 'WN']
check_depDelay_airports = df1[(df1['UniqueCarrier'] == carrier)]['DepDelay'].groupby(df1['Origin']).apply(get_stats).unstack()
check_airports.sort_values('Count', ascending = False, inplace = True)
check_airports[-5:]
#carriers.loc[carriers['Code'] == 'WN']

,Count,Max,Mean,Min
Origin,,,,
MAF,3852.0,592.0,8.359813,-10.0
IAD,3852.0,417.0,9.636812,-10.0
RSW,3828.0,368.0,7.901254,-16.0
JAN,3031.0,560.0,8.879578,-10.0
CRP,1886.0,351.0,6.981972,-10.0


In [21]:
def get_flight_depDelays(df, carrier, id_airport, extrem_values = False):
    df2 = df[(df['UniqueCarrier'] == carrier) & (df['Origin'] == id_airport)]
    # remove extreme values before fitting
    if extrem_values:
        df2['DepDelay'] = df2['DepDelay'].apply(lambda x:x if x < 60 else np.nan)
        df2.dropna(how = 'any')
    test2 = df2['DepDelay'].groupby(df2['CRSDepTime']).apply(get_stats).unstack()
    test2.reset_index(inplace=True)
    
    #htm = lambda x:x.hour*3600+x.minute*60+x.second
    test2.reset_index(inplace=True)
#     test2['CRSDep_Time_In_Min'] = test2['CRSDepTime'].apply(htm)
    return test2
def get_flights_arrDelays
def get_airport_info(code):
    return airports_abbr_dict[code]
def get_flight_info(code):
    return carriers_abbr_dict[code]

In [22]:
get_flight_delays(df1, 'WN', 'MAF')

KeyError: ('DepDelay', 'ArrDelay')

In [15]:
get_airport_info('MAF')

{'airport': 'Midland International',
 'city': 'Midland',
 'country': 'USA',
 'lat': 31.94252778,
 'long': -102.20191390000001,
 'state': 'TX'}

In [18]:
get_flight_info('WN')

'Southwest Airlines Co.'

### Model 1: One Airline, Multilple Airports

In [19]:

train = df1.values
#test = df1[df1['CRSDep_Date_Time'].apply(lambda x:x.date()) > datetime.date(2008, 9, 1)]


In [20]:
train

array([['WN', 'IAD', 'TPA', ..., 150.0, 128.0, 810],
       ['WN', 'IAD', 'TPA', ..., 145.0, 128.0, 810],
       ['WN', 'IND', 'BWI', ..., 90.0, 96.0, 515],
       ..., 
       ['DL', 'PBI', 'ATL', ..., 116.0, 121.0, 545],
       ['DL', 'IAD', 'ATL', ..., 117.0, 115.0, 533],
       ['DL', 'SAT', 'ATL', ..., 135.0, 123.0, 874]], dtype=object)

In [79]:
def get_delays_info(df, carrier):
    airports = df[df['UniqueCarrier'] == carrier]['Origin'].unique()
    i = 0
    cols = ['Airports', 'DepDelay', 'Mean']
    for airport in airports:
        test2 = get_flight_delays(df, carrier, airport, True)
        test2.loc[:, 'Airports'] = airport
        test2 = test2[cols]
        test2.dropna(how = 'any', inplace = True)
        if i == 0:
            merged_df = test2.copy()
        else:
            merged_df = pd.concat([merged_df, test2], ignore_index = True)
        i += 1    
    return merged_df

In [80]:
carrier = 'WN'
merged_df = get_delays_info(df2, carrier)
#merged_df.loc[:,:]

/Users/trininh/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/trininh/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/trininh/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if

In [ ]:
merged_df.shape

In [81]:
##Label Encoding
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(merged_df['Airports'])
#__________________________________________________________
# correspondance between the codes and tags of the airports
zipped = zip(integer_encoded, merged_df['Airports'])
label_airports = list(set(list(zipped)))
label_airports.sort(key = lambda x:x[0])
#label_airports

In [83]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
b = np.array(merged_df['CRSDep_Time_In_Min'])
b = b.reshape(len(b),1)
X = np.hstack((onehot_encoded, b))
Y = np.array(merged_df['Mean'])
Y = Y.reshape(len(Y), 1)

In [39]:
Y.shape

(7979, 1)

In [84]:
lm = linear_model.LinearRegression()
model = lm.fit(X,Y)
predictions = lm.predict(X)
print("MSE =", metrics.mean_squared_error(predictions, Y))

('MSE =', 17.820050905255339)


In [85]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [86]:
from sklearn.metrics import mean_squared_error, r2_score
poly = PolynomialFeatures(degree = 2)
reg = linear_model.LinearRegression()
X_ = poly.fit_transform(X_train)
reg.fit(X_, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [87]:
X_0 = poly.fit_transform(X_test)
pred = reg.predict(X_0)
print("MSE = ", metrics.mean_squared_error(pred, Y_test))

('MSE = ', 17.186905592657137)


In [88]:
from sklearn.metrics import mean_squared_error, r2_score
poly = PolynomialFeatures(degree = 2)
reg = linear_model.Ridge(alpha=.5)
X_ = poly.fit_transform(X_train)
reg.fit(X_, Y_train)

/Users/trininh/anaconda/lib/python2.7/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 2.10670104858e-17
  ' condition number: {}'.format(rcond), RuntimeWarning)


Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [89]:
X_0 = poly.fit_transform(X_test)
pred = reg.predict(X_0)
print("MSE = ", metrics.mean_squared_error(pred, Y_test))

('MSE = ', 17.177408904206839)


In [41]:
score_min = 10000
for pol_order in range(1, 3):
    for alpha in range(0, 20, 2):
        ridgereg = linear_model.Ridge(alpha = alpha/10, normalize=True)
        poly = PolynomialFeatures(degree = pol_order)
        #regr = linear_model.LinearRegression()
        X_ = poly.fit_transform(X_train)
        ridgereg.fit(X_, Y_train)        
        X_ = poly.fit_transform(X_test)
        result = ridgereg.predict(X_)
        score = metrics.mean_squared_error(result, Y_test)        
        if score < score_min:
            score_min = score
            parameters = [alpha/10, pol_order]
        print("n={} alpha={} , MSE = {:<0.5}".format(pol_order, alpha, score))

n=1 alpha=0 , MSE = 19.372
n=1 alpha=2 , MSE = 19.372
n=1 alpha=4 , MSE = 19.372
n=1 alpha=6 , MSE = 19.372
n=1 alpha=8 , MSE = 19.372
n=1 alpha=10 , MSE = 21.616
n=1 alpha=12 , MSE = 21.616
n=1 alpha=14 , MSE = 21.616
n=1 alpha=16 , MSE = 21.616
n=1 alpha=18 , MSE = 21.616
n=2 alpha=0 , MSE = 19.326
n=2 alpha=2 , MSE = 19.326
n=2 alpha=4 , MSE = 19.326
n=2 alpha=6 , MSE = 19.326
n=2 alpha=8 , MSE = 19.326
n=2 alpha=10 , MSE = 20.363
n=2 alpha=12 , MSE = 20.363
n=2 alpha=14 , MSE = 20.363
n=2 alpha=16 , MSE = 20.363
n=2 alpha=18 , MSE = 20.363


In [46]:
df3 = test
df3[:5]

,UniqueCarrier,Origin,Dest,CRSDep_Date_Time,DepTime_Formatted,DepDelay,CRSArrTime_Formatted,ArrTime_Formatted,ArrDelay,CRSElapsedTime,ActualElapsedTime
2458438,WN,ABQ,AMA,2008-05-24 18:30:00,18:27:00,-3.0,20:30:00,20:23:00,-7.0,60.0,56.0
2458439,WN,ABQ,AMA,2008-05-24 20:10:00,20:05:00,-5.0,22:10:00,21:55:00,-15.0,60.0,50.0
2458440,WN,ABQ,BWI,2008-05-24 14:25:00,14:58:00,33.0,20:10:00,20:15:00,5.0,225.0,197.0
2458441,WN,ABQ,BWI,2008-05-24 10:45:00,10:49:00,4.0,16:30:00,16:42:00,12.0,225.0,233.0
2458442,WN,ABQ,DAL,2008-05-24 15:00:00,16:12:00,72.0,17:45:00,18:45:00,60.0,105.0,93.0


### Testing the model with the test set (flights after May 28, 2008)

In [48]:
test

,UniqueCarrier,Origin,Dest,CRSDep_Date_Time,DepTime_Formatted,DepDelay,CRSArrTime_Formatted,ArrTime_Formatted,ArrDelay,CRSElapsedTime,ActualElapsedTime
2458438,WN,ABQ,AMA,2008-05-24 18:30:00,18:27:00,-3.0,20:30:00,20:23:00,-7.0,60.0,56.0
2458439,WN,ABQ,AMA,2008-05-24 20:10:00,20:05:00,-5.0,22:10:00,21:55:00,-15.0,60.0,50.0
2458440,WN,ABQ,BWI,2008-05-24 14:25:00,14:58:00,33.0,20:10:00,20:15:00,5.0,225.0,197.0
2458441,WN,ABQ,BWI,2008-05-24 10:45:00,10:49:00,4.0,16:30:00,16:42:00,12.0,225.0,233.0
2458442,WN,ABQ,DAL,2008-05-24 15:00:00,16:12:00,72.0,17:45:00,18:45:00,60.0,105.0,93.0
2458443,WN,ABQ,DAL,2008-05-24 07:00:00,07:04:00,4.0,09:40:00,09:37:00,-3.0,100.0,93.0
2458444,WN,ABQ,DAL,2008-05-24 18:00:00,18:13:00,13.0,20:40:00,20:49:00,9.0,100.0,96.0
2458445,WN,ABQ,DAL,2008-05-24 19:20:00,19:24:00,4.0,22:00:00,21:58:00,-2.0,100.0,94.0
2458446,WN,ABQ,DAL,2008-05-24 13:10:00,13:09:00,-1.0,15:50:00,15:38:00,-12.0,100.0,89.0
2458447,WN,ABQ,DAL,2008-05-24 10:05:00,10:05:00,0.0,12:45:00,12:38:00,-7.0,100.0,93.0


### Testing models with the test set (flights from January 2017)

In [90]:
test1 = pd.read_csv('../data/Nov-2015.csv')
test1 = test1.loc[:, ~data0_df.columns.str.contains('^Unnamed')]

In [92]:
test1

,YEAR,MONTH,DAY_OF_MONTH,UNIQUE_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME
0,2015,11,25,MQ,BTR,DFW,1340,1332.0,-8.0,1515,1454.0,-21.0,95.0,82.0,70.0
1,2015,11,27,MQ,BTR,DFW,1340,1335.0,-5.0,1515,1457.0,-18.0,95.0,82.0,61.0
2,2015,11,28,MQ,BTR,DFW,1340,1338.0,-2.0,1515,1456.0,-19.0,95.0,78.0,58.0
3,2015,11,29,MQ,BTR,DFW,1340,1334.0,-6.0,1515,1502.0,-13.0,95.0,88.0,62.0
4,2015,11,30,MQ,BTR,DFW,1340,1330.0,-10.0,1515,1457.0,-18.0,95.0,87.0,69.0
5,2015,11,1,MQ,DFW,BTR,1155,1147.0,-8.0,1320,1308.0,-12.0,85.0,81.0,59.0
6,2015,11,2,MQ,DFW,BTR,1155,1250.0,55.0,1320,1417.0,57.0,85.0,87.0,60.0
7,2015,11,3,MQ,DFW,BTR,1155,1208.0,13.0,1320,1348.0,28.0,85.0,100.0,59.0
8,2015,11,4,MQ,DFW,BTR,1155,1152.0,-3.0,1320,1326.0,6.0,85.0,94.0,54.0
9,2015,11,5,MQ,DFW,BTR,1150,1153.0,3.0,1314,1328.0,14.0,84.0,95.0,58.0


In [91]:
merged_df_test = get_delays_info(test1, carrier)

KeyError: 'UniqueCarrier'

In [49]:
merged_df_test[:5]

,Airports,CRSDep_Time_In_Min,mean
0,ABQ,21600,-0.297297
1,ABQ,21900,-2.583333
2,ABQ,22200,-0.035971
3,ABQ,22500,-0.746667
4,ABQ,22800,-0.733333


In [51]:
label_conversion = dict()
for s in label_airports:
    label_conversion[s[1]] = s[0]
merged_df_test['Airports'].replace(label_conversion, inplace = True)
for index, label in label_airports:
    temp = merged_df_test['Airports'] == index
    temp = temp.apply(lambda x:1.0 if x else 0.0)
    if index == 0:
        matrix = np.array(temp)
    else:
        matrix = np.vstack((matrix, temp))
matrix = matrix.T
b = np.array(merged_df_test['CRSDep_Time_In_Min'])
b = b.reshape(len(b),1)
X_test = np.hstack((matrix, b))
Y_test = np.array(merged_df_test['mean'])
Y_test = Y_test.reshape(len(Y_test), 1)

In [57]:
X_ = poly.fit_transform(X_test)
result = reg.predict(X_)
mse = metrics.mean_squared_error(result, Y_test)
'MSE = {:.2f}'.format(score)

'MSE = 24.57'

In [58]:
'Average Delay = {:.2f} min'.format(np.sqrt(mse))

'Average Delay = 4.96 min'

In [22]:
import matplotlib.pyplot as plt
tips = pd.DataFrame()
tips["prediction"] = pd.Series([float(s) for s in result]) 
tips["original_data"] = pd.Series([float(s) for s in Y]) 
sns.jointplot(x="original_data", y="prediction", data=tips, size = 6, ratio = 7,
              joint_kws={'line_kws':{'color':'yellow'}}, kind='reg', color='g')
plt.xlabel('Mean delays (min)', fontsize = 15)
plt.ylabel('Predictions (min)', fontsize = 15)
plt.plot(list(range(-10,25)), list(range(-10,25)), linestyle = ':', color = 'r')
sns.plt.show()

NameError: name 'result' is not defined

In [49]:
data0_df = pd.read_csv('../data/Nov-2015.csv')
data0_df = data0_df.loc[:, ~data0_df.columns.str.contains('^Unnamed')]

In [50]:
data0_df[:5]

,YEAR,MONTH,DAY_OF_MONTH,UNIQUE_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME
0,2015,11,25,MQ,BTR,DFW,1340,1332.0,-8.0,1515,1454.0,-21.0,95.0,82.0,70.0
1,2015,11,27,MQ,BTR,DFW,1340,1335.0,-5.0,1515,1457.0,-18.0,95.0,82.0,61.0
2,2015,11,28,MQ,BTR,DFW,1340,1338.0,-2.0,1515,1456.0,-19.0,95.0,78.0,58.0
3,2015,11,29,MQ,BTR,DFW,1340,1334.0,-6.0,1515,1502.0,-13.0,95.0,88.0,62.0
4,2015,11,30,MQ,BTR,DFW,1340,1330.0,-10.0,1515,1457.0,-18.0,95.0,87.0,69.0


In [8]:
data0_df.drop(['FLIGHTS'],axis=1, inplace=True)

In [16]:
data0_df

,YEAR,MONTH,DAY_OF_MONTH,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,Unnamed: 17
0,2017,1,7,AA,19805,AA,BOS,LAX,1850,1857.0,7.0,2241,2309.0,28.0,411.0,432.0,NaN
1,2017,1,8,AA,19805,AA,BOS,LAX,1850,1854.0,4.0,2241,2248.0,7.0,411.0,414.0,NaN
2,2017,1,9,AA,19805,AA,BOS,LAX,1850,1917.0,27.0,2242,2311.0,29.0,412.0,414.0,NaN
3,2017,1,10,AA,19805,AA,BOS,LAX,1903,1857.0,-6.0,2255,2306.0,11.0,412.0,429.0,NaN
4,2017,1,11,AA,19805,AA,BOS,LAX,1903,1856.0,-7.0,2255,2320.0,25.0,412.0,444.0,NaN
5,2017,1,12,AA,19805,AA,BOS,LAX,1903,1900.0,-3.0,2255,2326.0,31.0,412.0,446.0,NaN
6,2017,1,13,AA,19805,AA,BOS,LAX,1850,1847.0,-3.0,2242,2247.0,5.0,412.0,420.0,NaN
7,2017,1,15,AA,19805,AA,BOS,LAX,1903,1855.0,-8.0,2255,2222.0,-33.0,412.0,387.0,NaN
8,2017,1,16,AA,19805,AA,BOS,LAX,1903,1901.0,-2.0,2255,2213.0,-42.0,412.0,372.0,NaN
9,2017,1,17,AA,19805,AA,BOS,LAX,1903,1853.0,-10.0,2255,2139.0,-76.0,412.0,346.0,NaN
